# Prompting the models to classify the statements

## GPT 3.5 and GPT 4.1

In [28]:
%%capture output
%run ./04_prompt_creation.ipynb

In [29]:
import pandas as pd

chunking = "256_20"
only_text = True

path = f"../data/dfs/{'only_text_' if only_text else ''}{chunking}/ReferenceErrorDetection_data_with_chunk_info.pkl"
print(path)

# read the dataframe from a pickle file
df = pd.read_pickle(path)

../data/dfs/only_text_256_20/ReferenceErrorDetection_data_with_chunk_info.pkl


In [30]:
df.head()

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Citation ID,Statement with Citation,Corrected Statement,...,Reference Article PDF Available,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Error Type,Added,Previously Partially Substantiated,Top_3_Chunk_IDs,Top_3_Chunk_Texts
0,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,cit001_1,Others have aimed to reduce irreversibility or...,Others have aimed to reduce irreversibility or...,...,Yes,No,Yes,Unsubstantiated,Irrelevant,Irrelevant,No,NaN,"[15e4928e-d5b4-4358-a37a-0e2eef963320, 4472c31...",[68. (8 ) σ 1 σ 17 σ 18 σ 31 σ 30 σ 41 σ 46 σ ...
1,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,cit001_2,Some researchers have also studied various hea...,Some researchers have also studied various hea...,...,Yes,No,Yes,Unsubstantiated,Irrelevant,Irrelevant,No,NaN,"[1815ea07-43b1-4b3f-bbf0-35876f1f2e2f, cfeb9f2...",[The evaluations of nanofluid thermo-physical ...
2,PubPeer,c002,10.1155/2022/4601350,Oxidative Potential and Nanoantioxidant Activi...,Yes,Yes,Chemistry,cit002_1,The relative content of total flavonoids in th...,The relative content of total flavonoids in th...,...,Yes,No,Yes,Unsubstantiated,Irrelevant,Irrelevant,No,NaN,"[3d0ce96f-8ffb-4347-b469-d35015b7ef3a, 00c7040...",[Note when patients have undergone different m...
3,PubPeer,c003,10.1155/2022/2408685,The Choice of Anesthetic Drugs in Outpatient H...,Yes,Yes,Medicine,cit003_1,Research has shown that remimazolam tosylate e...,Research has shown that remimazolam tosylate e...,...,Yes,No,Yes,Unsubstantiated,Irrelevant,Irrelevant,No,NaN,"[c836b980-93f4-4dff-ac29-1735fa77227b, 0b41ff6...",[The second study analyzed 325 patients with 1...
4,PubPeer,c004,10.1155/2022/4783847,A Fault-Tolerant Structure for Nano-Power Comm...,Yes,Yes,Engineering,cit004_1,if the efficiency of the routing algorithm is ...,If the efficiency of the routing algorithm is ...,...,Yes,No,Yes,Unsubstantiated,Irrelevant,Irrelevant,No,NaN,"[1ecb8356-826f-491a-984a-722919362c0e, c0e8850...","[In the table, China's intermediary centrality..."


### Prompting the models (batch processing)

In [31]:
import os
import json

def create_batch_files(df, model, number_files=1, ignore_ids=[]):
    output_dir = f"../data/batch_files/{'only_text_' if only_text else ''}{chunking}/{model}"
    # Empty the folder if it exists
    if os.path.exists(output_dir):
        for filename in os.listdir(output_dir):
            file_path = os.path.join(output_dir, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    os.makedirs(output_dir, exist_ok=True)

    output_files = []
    for i in range(number_files):
        output_file = os.path.join(output_dir, f"prompt_batch_{i}.jsonl")
        # If the file already exists, empty it
        open(output_file, "w").close()
        output_files.append(output_file)
    
    for index, row in df.iterrows():
        if row['Reference Article Downloaded'] == 'Yes' and index not in ignore_ids:
            prompt = create_prompt(row)
            json_sequence = {
                "custom_id": f"request-{index}", 
                "method": "POST", 
                "url": "/v1/chat/completions", 
                "body": {
                    "model": model, 
                    "messages": [
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    "temperature": 0,
                }
            }

            output_file = output_files[index % number_files]
            with open(output_file, "a") as f:
                f.write(json.dumps(json_sequence) + "\n")
                
    # Remove empty output files from list
    output_files = [file for file in output_files if os.path.getsize(file) > 0]
    
    return output_files

In [32]:
import json

models = ["gpt-3.5-turbo-0125", "gpt-4.1-nano-2025-04-14", "gpt-4.1-mini-2025-04-14", "gpt-4.1-2025-04-14"]
model = models[3]

os.makedirs(f"../data/batch_responses/{'only_text_' if only_text else ''}{chunking}", exist_ok=True)
responses_dict_path = f"../data/batch_responses/{'only_text_' if only_text else ''}{chunking}/{model}_responses_dict_batch.json"

responses_dict = {}
try:
    with open(responses_dict_path, 'r') as file:
        responses_dict = json.load(file)
    ids_to_ignore = [int(key) for key in responses_dict.keys()]
except FileNotFoundError:
    ids_to_ignore = []

print(ids_to_ignore)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 2

In [33]:
batch_file_paths = create_batch_files(df, model, 5, ids_to_ignore)
batch_file_paths

[]

In [20]:
# Read the content of open_ai_key.txt into a variable
with open('../open_ai_key.txt', 'r') as file:
    open_ai_key = file.read().strip()

In [21]:
def check_batch(batch_id, client):
    batch = client.batches.retrieve(batch_id)
    print(f"{batch_id} - Current status: {batch.status}")
    
    if batch.status == 'completed' or batch.status == 'failed':
        return batch
    return None

In [22]:
import time
from openai import OpenAI

batch_input_files = []
batch_creation_responses = []
batches = []

client = OpenAI(api_key=open_ai_key)

def prompt_model_in_batches():
    global batch_input_files
    global batch_creation_responses
    global batches

    for batch_file_path in batch_file_paths:
        # Creating input file
        if os.stat(batch_file_path).st_size == 0:
            print(f"Skipping empty file: {batch_file_path}")
            continue
        batch_input_file = client.files.create(
            file=open(batch_file_path, "rb"),
            purpose="batch"
        )
        print(batch_input_file)
        batch_input_files.append(batch_input_file)

        # Starting batch job
        batch_input_file_id = batch_input_file.id
        batch_creation_response = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print("Started: " + batch_creation_response.id)

        time.sleep(5)
        # Check the status of the created batch until it is completed
        while True:
            batch_id = batch_creation_response.id
            batch = check_batch(batch_id, client)
            if batch:
                if batch.status == "failed":
                    return
                elif batch.status == "completed":
                    batches.append(batch)
                    break
            time.sleep(20)

GPT 3.5 Turbo:
- 256_20 only_text: 27:22 min
- 1024_20: 10:10 
- 1024_20 only_text: 34 min (and batch 8 and 9 not successfull due to token limit)

GPT 4.1 Nano:
- 256_20: 35:10 min
- 256_20 only_text: 22:40 min
- 1024_20: 15:55 min

GPT 4.1 Mini:
- 256_20: 19:50 min
- 256_20 only_text: 21:40 min
- 1024_20: 28:26 min
- 1024_20 only_text: 28:20 min (and batch 8 and 9 not successfull due to token limit)

GPT 4.1

In [23]:
%%time

prompt_model_in_batches()

FileObject(id='file-VsFPjX7kr4wxPt11XJQivk', bytes=262142, created_at=1752156301, filename='prompt_batch_3.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Started: batch_686fc88df210819081f8054fe973ef2b
batch_686fc88df210819081f8054fe973ef2b - Current status: in_progress
batch_686fc88df210819081f8054fe973ef2b - Current status: in_progress
batch_686fc88df210819081f8054fe973ef2b - Current status: in_progress
batch_686fc88df210819081f8054fe973ef2b - Current status: in_progress
batch_686fc88df210819081f8054fe973ef2b - Current status: in_progress
batch_686fc88df210819081f8054fe973ef2b - Current status: completed
FileObject(id='file-4WVBCEUBXdjgcywaMuMgst', bytes=251742, created_at=1752156409, filename='prompt_batch_4.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Started: batch_686fc8f9a6e881909af4eeeb4ed613cb
batch_686fc8f9a6e881909af4eeeb4ed613cb - Current status: in_progress
batch_686fc8f

### Check all open batches

In [11]:
import time
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

current_millis = int(time.time())
recently = current_millis - 1 * 60 * 60

open_batches = client.batches.list()
relevant_open_batches = [batch for batch in open_batches if batch.created_at >= recently]

In [12]:
list(relevant_open_batches)

[Batch(id='batch_6868096614388190a022996f6be9a17b', completion_window='24h', created_at=1751648614, endpoint='/v1/chat/completions', input_file_id='file-UfAtZGcxmckxTpS1aAMXJ5', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4.1-mini-2025-04-14 in organization org-6SCiN9rjR6tU38WJ0DavgNRs. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1751735014, failed_at=1751648615, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)),
 Batch(id='batch_686808c76bc081908c33aa7cf8e00f6f', completion_window='24h', created_at=1751648455, endpoint='/v1/chat/completions', input_file_id='file-CGFXQgJ9pC8oCg6VqtNFKD', object='b

In [80]:
client.files.content(relevant_open_batches[0].output_file_id).text

'{"id": "batch_req_68678a072b3881908b17d5aff3211ac4", "custom_id": "request-0", "response": {"status_code": 200, "request_id": "89ad89cfcb2a16e6f4cfc4ba768a5351", "body": {"id": "chatcmpl-BpVcqfN3e1zWesxuClqhceQMoxQyt", "object": "chat.completion", "created": 1751615996, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\n    \\"label\\": \\"Unsubstantiated\\",\\n    \\"explanation\\": \\"The reference article does not support the statement about reducing irreversibility or optimizing energy-consumed devices, as it focuses on fault analysis of three-phase induction motors.\\"\\n}", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 1163, "completion_tokens": 51, "total_tokens": 1214, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_toke

In [ ]:
# client.batches.cancel("batch_67e3cf592eb081908cd64e5e1dc55fa0")

ConflictError: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### Check the batch status

In [40]:
import time

def wait_for_batch_completion(batch_id, client, interval=10):
    while True:
        batch = check_batch(batch_id, client)
        if batch != None:
            return batch
        time.sleep(interval)

In [41]:
batch = wait_for_batch_completion("batch_68678417bf4c819085e0d77c012d5a7a", client)

batch_68678417bf4c819085e0d77c012d5a7a - Current status: completed


In [81]:
batch = check_batch("batch_686789f989bc8190b9cfc3a12dc8ccdc", client)

batch_686789f989bc8190b9cfc3a12dc8ccdc - Current status: completed


In [82]:
batches = [batch]

In [49]:
print(batch)

Batch(id='batch_68678417bf4c819085e0d77c012d5a7a', completion_window='24h', created_at=1751614487, endpoint='/v1/chat/completions', input_file_id='file-TubVrLSjHxewNncTaHNShi', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1751614569, error_file_id='file-39tX7oztfkdnqiNEpYDUMb', errors=None, expired_at=None, expires_at=1751700887, failed_at=None, finalizing_at=1751614556, in_progress_at=1751614490, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=25, total=25))


### Save model_responses

In [24]:
import json

# save responds of completed batches
for batch in batches:
    if batch.status != "completed":
        continue
    model_responses = client.files.content(batch.output_file_id).text

    # Parse the model_responses into a list of objects
    responses_list = [json.loads(line) for line in model_responses.splitlines()]
    # print(responses_list)

    try:
        for response in responses_list:
            responses_dict[int(response['custom_id'].split('-')[1])] = response
            responses_dict = dict(sorted(responses_dict.items(), key=lambda item: int(item[0])))
    except NameError:
        responses_dict = {int(response['custom_id'].split('-')[1]): response for response in responses_list}

In [25]:
import json

# Save responses_dict to a JSON file
with open(responses_dict_path, 'w') as file:
    json.dump(responses_dict, file, indent=4)

#### Save responds to Dataframe

In [26]:
# Create a new column in the dataframe to store the responses
if 'Model Classification' not in df.columns:
    df['Model Classification'] = None

# Iterate through the dataframe
for index, row in df.iterrows():
    if row['Reference Article Downloaded'] == 'Yes':
        i = index
        if i not in responses_dict:
            i = str(i)
        model_response = responses_dict[i]['response']['body']['choices'][0]['message']['content']
        
        # Save the response to the new column
        df.at[index, 'Model Classification'] = model_response

In [27]:
dfs_path = f"../data/dfs/{'only_text_' if only_text else ''}{chunking}/{model}/"
os.makedirs(dfs_path, exist_ok=True)
df.to_pickle(f"{dfs_path}ReferenceErrorDetection_data_with_prompt_results.pkl")
df.to_excel(f"{dfs_path}ReferenceErrorDetection_data_with_prompt_results.xlsx", index=False)